In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [3]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters, dtype=int)
        zero[idx]=1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [4]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [7]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [8]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1, :]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output
        
    total_loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(total_loss)

tensor(2.3238, grad_fn=<AddBackward0>)
tensor(0.8877, grad_fn=<AddBackward0>)
tensor(0.5534, grad_fn=<AddBackward0>)
tensor(0.3400, grad_fn=<AddBackward0>)
tensor(0.2190, grad_fn=<AddBackward0>)
tensor(0.1667, grad_fn=<AddBackward0>)
tensor(0.1278, grad_fn=<AddBackward0>)
tensor(0.1032, grad_fn=<AddBackward0>)
tensor(0.0863, grad_fn=<AddBackward0>)
tensor(0.0800, grad_fn=<AddBackward0>)
tensor(0.0651, grad_fn=<AddBackward0>)
tensor(0.0710, grad_fn=<AddBackward0>)
tensor(0.0526, grad_fn=<AddBackward0>)
tensor(0.0456, grad_fn=<AddBackward0>)
tensor(0.0488, grad_fn=<AddBackward0>)
tensor(0.0389, grad_fn=<AddBackward0>)
tensor(0.0337, grad_fn=<AddBackward0>)
tensor(0.0302, grad_fn=<AddBackward0>)
tensor(0.0485, grad_fn=<AddBackward0>)
tensor(0.0312, grad_fn=<AddBackward0>)
tensor(0.0275, grad_fn=<AddBackward0>)
tensor(0.0352, grad_fn=<AddBackward0>)
tensor(0.0252, grad_fn=<AddBackward0>)
tensor(0.0218, grad_fn=<AddBackward0>)
tensor(0.0198, grad_fn=<AddBackward0>)
tensor(0.0183, grad_fn=<A

In [9]:
start = torch.zeros(1, len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
print(output_string)

hellorememnme eme e e e e e e e e e e e e e e e 
